In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [2]:
def encodingByMean(values, columnData, setDatos):
    df = pd.DataFrame()
    column = columnData
    if(setDatos=='set_train'):
        df[values] = pd.get_dummies(set_train[column])
    else:
        df[values] = pd.get_dummies(test_values[column])
    df['sumatory'] = 0
    for i in range(len(values)):
        df['sumatory'] += df[values[i]] * df[values[i]].mean()
    
    return df['sumatory']

In [3]:
train_values = pd.read_csv('train_values.csv')
train_labels = pd.read_csv('train_labels.csv')
test_values = pd.read_csv('test_values.csv',index_col='building_id')

In [4]:
set_train = train_values.merge(train_labels)
set_train.drop( columns=["building_id"], inplace=True)
set_train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3


Nos quedamos con las construcciones con una cantidad de pisos igual o menor a seis

In [5]:
set_train = set_train[set_train['count_floors_pre_eq'] <= 6]

In [6]:
drop_columns = set_train.columns.values[26:38]
set_train.drop(columns= drop_columns, inplace = True)

In [7]:
set_train['base_condition'] = encodingByMean(['h', 'i', 'r', 'u', 'w'] , 'foundation_type', 'set_train') + encodingByMean(['n', 'o', 't'] , 'land_surface_condition', 'set_train')
test_values['base_condition'] = encodingByMean(['h', 'i', 'r', 'u', 'w'] , 'foundation_type', 'test_values') + encodingByMean(['n', 'o', 't'] , 'land_surface_condition', 'test_values')

In [8]:
set_train['base_condition'] = set_train['land_surface_condition'] + set_train['foundation_type']

In [9]:
materials = set_train.iloc[:,14:25]
materials['damage_grade'] = set_train['damage_grade']

materials_name  = ['has_superstructure_rc_engineered', 'has_superstructure_rc_non_engineered', 'has_superstructure_cement_mortar_brick',
               'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick', 'has_superstructure_mud_mortar_stone',
               'has_superstructure_stone_flag', 'has_superstructure_timber', 'has_superstructure_adobe_mud', 
               'has_superstructure_bamboo', 'has_superstructure_other']

values = [1, 0.9, 0.85, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05]
    
for m in range(11):
    set_train.loc[:, [materials_name[m]]] = set_train.loc[:, [materials_name[m]]]*values[m]
    
set_train['resistance'] = 0
for n in range(11):
    set_train['resistance'] += set_train[materials_name[n]]

In [10]:
for m in range(11):
    test_values.loc[:, [materials_name[m]]] = test_values.loc[:, [materials_name[m]]]*values[m]
    
test_values['resistance'] = 0
for n in range(11):
    test_values['resistance'] += test_values[materials_name[n]]

In [11]:
features = ['age',
            'geo_level_1_id',
            'area_percentage',
            'count_floors_pre_eq',
            'geo_level_2_id',
            'height_percentage',
            'resistance',
            'base_condition',
            'damage_grade']

In [12]:
set_train[features].to_csv('data_set.csv', index=False)
test_values.to_csv('data_test.csv')